## Capacitated Vehicle Routing Problem

$\begin{array}{lllll}
\displaystyle \textbf{Minimize}      & \displaystyle \sum_{k \in K}\sum_{(i,j) \in E}{c_{ij}x_{ij}^k}     &&    & (1)                                                                               \\
\displaystyle \textbf{Subject To}    & \displaystyle \sum_{k \in K}\sum_{i \in V, i\neq j}{x_{ij}^k} = 1    && \forall j \in V - \{0\}  & (2)                                                                               \\
                                    & \displaystyle \sum_{j \in V-\{0\}}{x_{0j}^k} = 1                               && \forall k \in K  & (3)                                                                               \\
                                    & \displaystyle \sum_{i \in V, i\neq j}{x_{ij}^k} - \sum_{i \in V}{x_{ji}^k} = 0 && \forall j \in V, \forall k \in K & (4) \\
                                    & \displaystyle \sum_{i \in V}\sum_{j \in V-\{0\}, i \neq j}{q_{ij} x_{ij}^k} \leq Q && \forall k \in K & (5) \\
                                    & \displaystyle \sum_{k \in K}\sum_{(i,j)\in S, i \neq j}{x_{ij}^k}\leq |S|-1 && S \subseteq V - \{0\} & (6) \\
                                    & x_{ij}^k \in \{0,1\} && \forall k \in K, \forall (i,j) \in E & (7) \\
                                    &                                               &&                  &                                                                                   \\
\displaystyle \textbf{Where}         && G(V,E)                                            & =                 & \textrm{A graph showing the location and route of the customer}             \\
                                    && V = \{0, 1,\ldots,n\}                             & =                 & \textrm{A collection of nodes. The node that represents the Depot is 0, and the nodes that represent each customer are 1 to }n                               \\
                                    && E                                            & =                 & \textrm{Set of Edges connecting each node }e_{ij} = (i,j)                     \\
                                    && K = \{1,2,\ldots,|K|\}                                            & =                 & \textrm{Set of Vehicles}                    \\
                                    && d_i \geq 0                                       & =                 & \textrm{Demand of customer } i   \\
                                    && Q \geq 0                                          & =                 & \textrm{Maximum vehicle capacity (all vehicles have the same maximum capacity Q)}                                                                \\
                                    && c_{ij}                                          & =   & \textrm{Travel costs between customer } i \textrm{ and } j\\
                                    && x_{ij}^k                                          & =                 & \begin{cases}
                                                                                                                   1, \textrm{if a vehicle } k \textrm{ has taken the route } e_{ij}                                \\
                                                                                                                   0, \textrm{otherwise}                                                            \\
                                                                                                                  \end{cases}
\end{array}$

In [3]:
import pandas as pd
import numpy as np
import pulp
import itertools

In [2]:
customer_count = 4
vehicle_capacity = 50
vehicle_count = 2
depot_point = ()
distance = [[]]

In [ ]:
for vechicle in range(1, vehicle_count+1):
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

    x = [[[pulp.LpVariable(f"x{i}{j}{k}", cat="Binary") if i != j else None 
                                                            for k in range(vehicle_count+1)] 
                                                            for j in range(customer_count)] 
                                                            for i in range(customer_count)]
    ## Objective function
    problem += pulp.lpSum(distance[i][j] * x[i][j][k] if i != j else 0 
                                                            for k in range(vehicle_count)
                                                            for j in range(customer_count)
                                                            for i in range(customer_count))
    
    ## Constraints
    ## Constraint 2
    for j in range(1, customer_count):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 
                                                for i in range(customer_count)
                                                 for k in range(vehicle_count)) == 1
    
    ## Constraint 3
    for k in range(vehicle_count):
        problem += pulp.lpSum(x[0][j][k] for j in range(1, customer_count)) == 1
        problem += pulp.lpSum(x[i][0][k] for i in range(1, customer_count)) == 1

    ## Constraint 4
    for k in range(vehicle_count):
        for j in range(customer_count):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0
                                                for i in range(customer_count)) - pulp.lpSum(x[j][i][k] for i in range(customer_count)) == 0
    
    ## Constraint 5
    for k in range(vehicle_count):
        problem += pulp.lpSum(df.demand[j] * x[i][j][k] if i != j else 0 
                                                            for i in range(customer_count)
                                                            for j in range(1, customer_count)) <= vehicle_capacity
    
    ## Constraint 6
    subtours = []
    for i in range(2, customer_count):
        subtours += itertools.combinations(range(1, customer_count), i)

    for s in subtours:
        problem += pulp.lpSum(x[i][j][k] if i != j else 0 for i, j in itertools.permutations(s,2) for k in range(vehicle_count)) <= len(s)-1


    if problem.solve() == 1:
        print("Vehicle Requirements:", vehicle_count)
        print("Vehicle Capacity:", vehicle_capacity)
        print("Distance:", pulp.value(problem.objective))